In [1]:
query = "insan öldürmenin cezası"

In [2]:
# Gokhan Has

import scipy
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

In [3]:
# Install BERT model by using SentenceTransformer package.
model = SentenceTransformer('dbmdz/bert-base-turkish-128k-cased')

In [4]:
# Read dataset function ...
def read_csv(filepath):
     if os.path.splitext(filepath)[1] != '.csv':
          return  # or whatever
     seps = [';', '\t']                    # ',' is default
     encodings = [None, 'utf-8', 'ISO-8859-1', 'utf-16','ascii']  # None is default
     for sep in seps:
         for encoding in encodings:
              try:
                  return pd.read_csv(filepath, encoding=encoding, sep=sep)
              except Exception:  # should really be more specific 
                  pass
     raise ValueError("{!r} is has no encoding in {} or seperator in {}"
                      .format(filepath, encodings, seps))

In [ ]:
# Get the dataset embeddings ...
input_df = read_csv('../datasets/exp1_dataset.csv')
input_df = input_df.head(4603)

for col in input_df.columns: 
    print(col) 

print(input_df.columns.tolist())

#sentences = list(input_df.groupby('SENTENCE').groups.items())
sentences = input_df['SENTENCE'].values.tolist()

sentence_embeddings = model.encode(sentences)

print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))

print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])

SENTENCE
LAW
['SENTENCE', 'LAW']


In [ ]:

queries = [query]
query_embeddings = model.encode(queries)

# Find the closest 3 sentences of the corpus for each query sentence based on cosine similarity
number_top_matches = 10 #@param {type: "number"}

print("Anlamsal arama sonuçları")

for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("ARANAN CÜMLE:", query)
    print("\nEN YÜKSEK 10 SONUÇ GÖSTERİLİYOR :\n")

    for idx, distance in results[0:number_top_matches]:
        print(sentences[idx].strip(), "(Cosine Score: %.4f)" % (1-distance))